In [16]:
import pandas as pd
import glob

In [17]:
def readfile(temp: str) -> pd.core.frame.DataFrame:
    temp = pd.read_excel(temp, sheet_name = 'Day')
    temp = temp.T
    temp.rename(columns=temp.iloc[0], inplace = True)
    temp = temp.drop([temp.index[0]])
    return temp

CAR table & FX table

In [18]:
def load_car_fx_df(date: str) -> pd.core.frame.DataFrame:
    #CAR
    path = 'resources/car_table/'
    car_files = glob.glob(path + "/*.csv")
    for item in car_files: car_files[car_files.index(item)] = item.split('/')[2].split('.')[0]
    car_files.sort()
    car_df = pd.DataFrame()
    for element in car_files:
        temp_df = pd.read_csv(f"resources/car_table/{element}.TW.csv", index_col=None, header=0)
        temp_df[["company_code"]] = f"{element}"
        temp_df = temp_df.loc[temp_df["Date"] == date].drop(["Unnamed: 0", "Date"], axis=1)
        temp_df = temp_df.set_index("company_code")
        car_df = pd.concat([car_df, temp_df])
    car_df = car_df[['abnormal_returns','t-1~t CAR', 't-2~t CAR', 't-3~t CAR', 't-3~t+1 CAR', 
                    't-3~t+2 CAR', 't-3~t+3 CAR', 't~t+1 CAR', 't~t+2 CAR', 't~t+3 CAR']]
    car_df.index.name = None

    #FX
    path = 'resources/fx_table/'
    fx_files = glob.glob(path + "/*")
    for item in fx_files: fx_files[fx_files.index(item)] = item.split('/')[2]
    fx_files.sort()
    fx_df = pd.DataFrame()
    for element in fx_files:
        temp_df = pd.read_csv(f"resources/fx_table/{element}/{element}.TW.csv", index_col=None, header=0)
        temp_df[["company_code"]] = f"{element}"
        temp_df = temp_df.loc[temp_df["Date"] == date].drop(["Unnamed: 0", "Date"], axis=1)
        temp_df = temp_df.set_index("company_code")
        fx_df = pd.concat([fx_df, temp_df])
    fx_df = fx_df[['FX factor']]
    fx_df.index.name = None

    #import company industry
    industry_type = pd.read_csv('resources/my_data.csv')
    industry_type = industry_type.set_index("Code")
    industry_type.index.name = None
    industry_type.index = industry_type.index.astype(object)
    industry_type.index = [f'{x}' for x in industry_type.index]

    #concat
    car_fx_df = pd.concat([car_df, fx_df], axis=1).dropna()
    car_fx_df = pd.concat([car_fx_df, industry_type], axis=1).dropna()

    return car_fx_df

E參數

In [19]:
def load_environmental_df(date: str) -> pd.core.frame.DataFrame:
    path = 'resources/E參數'
    files = glob.glob(path + "/*.xlsx")
    for item in files: files[files.index(item)] = item.split('/')[2].split('.')[0]
    df = pd.DataFrame()
    for item in files: 
        try:
            df[f'{item}'] = readfile(f'resources/E參數/{item}.xlsx').fillna(0).iloc[0:][[date]]
        except KeyError:
            print(f"{item} occurs error")
    if df.empty: 
        print("load_environmental_df is null")
        return df
    df.index = df.index.str.split().str[0].astype('int64')
    environmental_df = df

    environmental_df.index = environmental_df.index.astype(object)
    environmental_df.index = [f'{x}' for x in environmental_df.index]
    
    return environmental_df

籌碼面

In [20]:
def load_chip_df(date: str) -> pd.core.frame.DataFrame:
    path = 'resources/籌碼面'
    files = glob.glob(path + "/*.xlsx")
    for item in files: files[files.index(item)] = item.split('/')[2].split('.')[0]
    df = pd.DataFrame()
    for item in files: 
        try:
            df[f'{item}'] = readfile(f'resources/籌碼面/{item}.xlsx').fillna(0).iloc[0:][[date]]
        except KeyError:
            print(f"{item} occurs error")
    if df.empty: 
        print("load_chip_df is null")
        return df
    df.index = df.index.str.split().str[0].astype('int64')
    chip_df = df

    chip_df.index = chip_df.index.astype(object)
    chip_df.index = [f'{x}' for x in chip_df.index]
    
    return chip_df

公司財務相關

In [21]:
def load_financial_df(date: str) -> pd.core.frame.DataFrame:
    path = 'resources/公司財務相關'
    files = glob.glob(path + "/*.xlsx")
    for item in files: files[files.index(item)] = item.split('/')[2].split('.')[0]
    df = pd.DataFrame()
    for item in files: 
        try:
            df[f'{item}'] = readfile(f'resources/公司財務相關/{item}.xlsx').iloc[1:][[date]]
        except KeyError:
            print(f"{item} occurs error")
    if df.empty: 
        print("load_financial_df is null")
        return df
    df.index = df.index.str.split().str[0].astype('int64')
    financial_df = df

    financial_df.index = financial_df.index.astype(object)
    financial_df.index = [f'{x}' for x in financial_df.index]   
     
    return financial_df

公司治理

In [22]:
def load_governence_df(date: str) -> pd.core.frame.DataFrame:
    path = 'resources/公司治理'
    files = glob.glob(path + "/*.xlsx")
    for item in files: files[files.index(item)] = item.split('/')[2].split('.')[0]
    df = pd.DataFrame()
    for item in files: 
        try:
            df[f'{item}'] = readfile(f'resources/公司治理/{item}.xlsx').iloc[0:][[date]]
        except KeyError:
            print(f"{item} occurs error")
    if df.empty: 
        print("load_governence_df is null")
        return df
    df.index = df.index.str.split().str[0].astype('int64')
    governence_df = df

    governence_df.index = governence_df.index.astype(object)
    governence_df.index = [f'{x}' for x in governence_df.index]

    return governence_df

公司年資

In [23]:
def load_company_age_df(date: str) -> pd.core.frame.DataFrame:
    df = pd.DataFrame()
    try:
        df["company age"] = readfile(f'resources/company age.xlsx').T.iloc[0:][[date]]
    except KeyError:
        return False
    if df.empty: 
        print("load_company_age_df is null")
        return df
    df.index = df.index.str.split().str[0].astype('int64')
    company_age_df = df

    company_age_df.index = company_age_df.index.astype(object)
    company_age_df.index = [f'{x}' for x in company_age_df.index]

    return company_age_df

控制型態

In [24]:
def load_control_type_df(date: str) -> pd.core.frame.DataFrame:
    df = pd.DataFrame()
    try:
        df["control type"] = readfile(f'resources/control type.xlsx').T.iloc[0:][[date]]
    except KeyError:
        return False
    if df.empty: 
        print("load_control_type_df is null")
        return df
    df.index = df.index.str.split().str[0].astype('int64')
    control_type_df = pd.get_dummies(df["control type"], dtype=int)

    control_type_df.index = control_type_df.index.astype(object)
    control_type_df.index = [f'{x}' for x in control_type_df.index]
    
    return control_type_df

Concat All and output

In [25]:
def concat_output(date: str):
    car_fx_df = load_car_fx_df(date)
    if car_fx_df.empty == True:
        print(f"The markets were closed on {date}")
        return
    environmental_df = load_environmental_df(date)
    chip_df = load_chip_df(date)
    financial_df = load_financial_df(date)
    governence_df = load_governence_df(date)
    company_age_df = load_company_age_df(date)
    control_type_df = load_control_type_df(date)
    final_output = pd.DataFrame()
    if car_fx_df.empty == False: final_output = pd.concat([final_output, car_fx_df], axis=1).dropna()
    if environmental_df.empty == False: final_output = pd.concat([final_output, environmental_df], axis=1).dropna()
    if chip_df.empty == False: final_output = pd.concat([final_output, chip_df], axis=1).dropna()
    if financial_df.empty == False: final_output = pd.concat([final_output, financial_df], axis=1).dropna()
    if governence_df.empty == False: final_output = pd.concat([final_output, governence_df], axis=1).dropna()
    if company_age_df.empty == False: final_output = pd.concat([final_output, company_age_df], axis=1).dropna()
    if control_type_df.empty == False: final_output = pd.concat([final_output, control_type_df], axis=1).dropna()
    final_output.to_csv(f"{date}.csv")

In [26]:
concat_output("2019-10-01")

Renewable energy usage rate of listed companies occurs error
Greenhouse gas emission intensity occurs error
load_environmental_df is null
